# 라이브러리 호출

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK # HyperOPT
from nltk.corpus import names # nltk
import nltk
nltk.download("names")
from nltk import NaiveBayesClassifier
from scipy import stats
from collections import Counter
import random
# for stacking ensemble
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# cv스태밍
from sklearn.metrics import mean_absolute_error


import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='batang')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="batang", rc={"axes.unicode_minus":False}, style='whitegrid')
plt.rc('figure', figsize=(10,8))

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


# 데이터로딩

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

In [3]:
test = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Test')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G,3.0,G3,S,G/3/S,TRAPPIST-1e,27,False,0,0,0,0,0,Nelly Carsoning,NaN
1,0018_01,Earth,False,F,4.0,F4,S,F/4/S,TRAPPIST-1e,19,False,0,9,0,2823,0,Lerome Peckers,NaN
2,0019_01,Europa,True,C,0.0,C0,S,C/0/S,55 Cancri e,31,False,0,0,0,0,0,Sabih Unhearfus,NaN
3,0021_01,Europa,False,C,1.0,C1,S,C/1/S,TRAPPIST-1e,38,False,0,6652,0,181,585,Meratz Caltilter,NaN
4,0023_01,Earth,False,F,5.0,F5,S,F/5/S,TRAPPIST-1e,20,False,10,0,635,0,0,Brence Harperez,NaN


In [6]:
data.drop('Cabin2', inplace=True, axis=1)

In [7]:
test.drop('Cabin2', inplace=True, axis=1)

In [8]:
data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin1', 'Combi', 'Cabin3',
       'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported'],
      dtype='object')

## 이상치 확인 및 제거

In [9]:
def outlier_detection_train(df, n, columns):
    rows = []
    will_drop_train = []
    for col in columns:
        Q1 = np.nanpercentile(data[col], 25)
        Q3 = np.nanpercentile(data[col], 75)
        IQR = Q3 - Q1
        outlier_point = 1.5 * IQR
        rows.extend(df[(df[col] < Q1 - outlier_point)|(df[col] > Q3 + outlier_point)].index)
    for r, c in Counter(rows).items():
        if c >= n: will_drop_train.append(r)
    return will_drop_train

In [10]:
will_drop_train = outlier_detection_train(data, 5 ,data.select_dtypes(["float", "int"]).columns)
will_drop_train

[338,
 1390,
 6469,
 7038,
 1936,
 3317,
 3980,
 4762,
 6509,
 7007,
 7065,
 7294,
 7689,
 7957,
 8064]

In [11]:
data.drop(will_drop_train, inplace = True, axis = 0)

## 새로운 feature 생성

### 총 사용금액, 그리고 사용한 금액에 따라 poor, middle, rich로 분류

In [12]:
data["Total"] = data["RoomService"] + data["FoodCourt"] + data["ShoppingMall"] + data["Spa"] +\
data["VRDeck"] 
data["RichPoor"] = data["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

test["Total"] = test["RoomService"] + test["FoodCourt"] + test["ShoppingMall"] + test["Spa"] +\
test["VRDeck"] 
test["RichPoor"] = test["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

### 그룹여행객 여부

In [13]:
data["GroupId"] = data["PassengerId"].apply(lambda x: x.split("_")[0])
test["GroupId"] = test["PassengerId"].apply(lambda x: x.split("_")[0])
data["GroupNo"] = data["PassengerId"].apply(lambda x: x.split("_")[1])
test["GroupNo"] = test["PassengerId"].apply(lambda x: x.split("_")[1])

train_g = data[data["GroupId"].duplicated()]["GroupId"]
test_g = test[test["GroupId"].duplicated()]["GroupId"]
data["Group"] = data["GroupId"].apply(lambda x: x in train_g.values)
test["Group"] = test["GroupId"].apply(lambda x: x in test_g.values)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8678 entries, 0 to 8692
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PassengerId   8678 non-null   object
 1   HomePlanet    8678 non-null   object
 2   CryoSleep     8678 non-null   bool  
 3   Cabin1        8575 non-null   object
 4   Combi         8575 non-null   object
 5   Cabin3        8575 non-null   object
 6   Cabin         8575 non-null   object
 7   Destination   8678 non-null   object
 8   Age           8678 non-null   int64 
 9   VIP           8678 non-null   bool  
 10  RoomService   8678 non-null   int64 
 11  FoodCourt     8678 non-null   int64 
 12  ShoppingMall  8678 non-null   int64 
 13  Spa           8678 non-null   int64 
 14  VRDeck        8678 non-null   int64 
 15  Name          8478 non-null   object
 16  Transported   8678 non-null   bool  
 17  Total         8678 non-null   int64 
 18  RichPoor      8678 non-null   object
 19  GroupI

### 나이브 베어스를 활용한 이름을 통한 성별 분류

In [15]:
# Train_Data
names_train_data = []
for n in data["Name"]:
    n = str(n)
    a = n.split()
    names_train_data.append(a[0])

#### 훈련셋

In [16]:
# 이름과 성 분리
names_train_data = []
for i in data["Name"]:
    i = str(i)
    a = i.split()
    names_train_data.append(a[0])

In [17]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [18]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [19]:
names_train_data

['Maham',
 'Juanna',
 'Altark',
 'Solam',
 'Willy',
 'Sandie',
 'Billex',
 'Candra',
 'Andona',
 'Erraiam',
 'Altardr',
 'Wezena',
 'Berers',
 'Reney',
 'Elle',
 'Justie',
 'Flats',
 'Carry',
 'Alus',
 'Lyde',
 'Philda',
 'Almary',
 'Glendy',
 'Mollen',
 'Breney',
 'Mael',
 'Terta',
 'nan',
 'Penton',
 'Karard',
 'Anyoni',
 'Ceros',
 'Ginia',
 'Coobix',
 'Cinets',
 'Dontch',
 'Ziba',
 'Luse',
 'Marina',
 'Loise',
 'Jorgie',
 'Margia',
 'Ankalik',
 'Jodye',
 'Kayne',
 'Cassa',
 'Zelowl',
 'Mass',
 'Sony',
 'Vivia',
 'Elaney',
 'Elson',
 'Okulas',
 'Instab',
 'Zinoces',
 'Warry',
 'Shanya',
 'Sterry',
 'nan',
 'Colatz',
 'Diandy',
 'Ninaha',
 'Celine',
 'Velyne',
 'Cinst',
 'nan',
 'Meremy',
 'Nelly',
 'Thell',
 'Gorn',
 'Aldibah',
 'Conk',
 'Pon',
 'Spuri',
 'Dellie',
 'Totse',
 'Eaturs',
 'nan',
 'Coren',
 'Furudah',
 'Jodye',
 'Stmeal',
 'Heremy',
 'Deanne',
 'Tinez',
 'Gracy',
 'Stald',
 'Tiney',
 'Alchium',
 'Doria',
 'Leence',
 'Aliey',
 'Thewis',
 'Book',
 'Ritany',
 'Arlen',
 'Sh

In [20]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [21]:
# 성별 feature 생성
names_gender = []
for i in names_train_data:
    names_gender.append(classifier.classify(gender_features(i)))

# create new column called 'gender'    
data["Gender"] = names_gender

In [22]:
data.Gender[data.Name.isna()] = 'female'

In [23]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

#### 테스트셋

In [24]:
# 이름과 성 분리
names_test_data = []
for i in test["Name"]:
    i = str(i)
    a = i.split()
    names_test_data.append(a[0])

In [25]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [26]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [27]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [28]:
# 성별 feature 생성
names_gender = []
for i in names_test_data:
    names_gender.append(classifier.classify(gender_features(i)))

In [29]:
test["Gender"] = pd.Series(names_gender)

In [30]:
test.Gender[test.Name.isna()] = 'female'

In [31]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

## Cabin 결측값들 제거

In [32]:
data.dropna(axis=0, inplace=True)

## 필요없는 features 제거

In [33]:
target = data['Transported']
data = data.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId",\
                  "GroupNo", "Transported"], axis = 1)
test = test.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId",\
                  "GroupNo", "Transported"], axis = 1)


## 째려보기

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8375 non-null   object
 1   CryoSleep     8375 non-null   bool  
 2   Cabin1        8375 non-null   object
 3   Cabin3        8375 non-null   object
 4   Destination   8375 non-null   object
 5   Age           8375 non-null   int64 
 6   VIP           8375 non-null   bool  
 7   RoomService   8375 non-null   int64 
 8   FoodCourt     8375 non-null   int64 
 9   ShoppingMall  8375 non-null   int64 
 10  Spa           8375 non-null   int64 
 11  VRDeck        8375 non-null   int64 
 12  RichPoor      8375 non-null   object
 13  Group         8375 non-null   bool  
 14  Gender        8375 non-null   object
dtypes: bool(3), int64(6), object(6)
memory usage: 875.1+ KB


In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    4277 non-null   object
 1   CryoSleep     4277 non-null   bool  
 2   Cabin1        4214 non-null   object
 3   Cabin3        4214 non-null   object
 4   Destination   4277 non-null   object
 5   Age           4277 non-null   int64 
 6   VIP           4277 non-null   bool  
 7   RoomService   4277 non-null   int64 
 8   FoodCourt     4277 non-null   int64 
 9   ShoppingMall  4277 non-null   int64 
 10  Spa           4277 non-null   int64 
 11  VRDeck        4277 non-null   int64 
 12  RichPoor      4277 non-null   object
 13  Group         4277 non-null   bool  
 14  Gender        4277 non-null   object
dtypes: bool(3), int64(6), object(6)
memory usage: 413.6+ KB


## 원핫인코딩

### boolean 타입 피처들 object로 캐스팅

In [36]:
bool_data = [column for column in data.select_dtypes(["bool"])]

bool_data

['CryoSleep', 'VIP', 'Group']

In [37]:
data["VIP"] = data["VIP"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["CryoSleep"] = data["CryoSleep"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["Group"] = data["Group"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8375 non-null   object
 1   CryoSleep     8375 non-null   object
 2   Cabin1        8375 non-null   object
 3   Cabin3        8375 non-null   object
 4   Destination   8375 non-null   object
 5   Age           8375 non-null   int64 
 6   VIP           8375 non-null   object
 7   RoomService   8375 non-null   int64 
 8   FoodCourt     8375 non-null   int64 
 9   ShoppingMall  8375 non-null   int64 
 10  Spa           8375 non-null   int64 
 11  VRDeck        8375 non-null   int64 
 12  RichPoor      8375 non-null   object
 13  Group         8375 non-null   object
 14  Gender        8375 non-null   object
dtypes: int64(6), object(9)
memory usage: 1.0+ MB


In [39]:
# test 셋
bool_test = [column for column in test.select_dtypes(["bool"])]

test["VIP"] = test["VIP"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
test["CryoSleep"] = test["CryoSleep"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
test["Group"] = test["Group"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    4277 non-null   object
 1   CryoSleep     4277 non-null   object
 2   Cabin1        4214 non-null   object
 3   Cabin3        4214 non-null   object
 4   Destination   4277 non-null   object
 5   Age           4277 non-null   int64 
 6   VIP           4277 non-null   object
 7   RoomService   4277 non-null   int64 
 8   FoodCourt     4277 non-null   int64 
 9   ShoppingMall  4277 non-null   int64 
 10  Spa           4277 non-null   int64 
 11  VRDeck        4277 non-null   int64 
 12  RichPoor      4277 non-null   object
 13  Group         4277 non-null   object
 14  Gender        4277 non-null   object
dtypes: int64(6), object(9)
memory usage: 501.3+ KB


### 더미화

In [40]:
# drop_first 첫번째 범주는 제거하고 더미화
# 다른 범주가 전부 0이면 자동적으로 첫번째 범주가 1인걸 알 수 있기에 사용
df = pd.get_dummies(data, drop_first = True)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Age                        8375 non-null   int64
 1   RoomService                8375 non-null   int64
 2   FoodCourt                  8375 non-null   int64
 3   ShoppingMall               8375 non-null   int64
 4   Spa                        8375 non-null   int64
 5   VRDeck                     8375 non-null   int64
 6   HomePlanet_Europa          8375 non-null   uint8
 7   HomePlanet_Mars            8375 non-null   uint8
 8   CryoSleep_Yes              8375 non-null   uint8
 9   Cabin1_B                   8375 non-null   uint8
 10  Cabin1_C                   8375 non-null   uint8
 11  Cabin1_D                   8375 non-null   uint8
 12  Cabin1_E                   8375 non-null   uint8
 13  Cabin1_F                   8375 non-null   uint8
 14  Cabin1_G                

In [42]:
# test
dt = pd.get_dummies(test, drop_first = True)
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Age                        4277 non-null   int64
 1   RoomService                4277 non-null   int64
 2   FoodCourt                  4277 non-null   int64
 3   ShoppingMall               4277 non-null   int64
 4   Spa                        4277 non-null   int64
 5   VRDeck                     4277 non-null   int64
 6   HomePlanet_Europa          4277 non-null   uint8
 7   HomePlanet_Mars            4277 non-null   uint8
 8   CryoSleep_Yes              4277 non-null   uint8
 9   Cabin1_B                   4277 non-null   uint8
 10  Cabin1_C                   4277 non-null   uint8
 11  Cabin1_D                   4277 non-null   uint8
 12  Cabin1_E                   4277 non-null   uint8
 13  Cabin1_F                   4277 non-null   uint8
 14  Cabin1_G                

### 스케일링

In [43]:
# 스케일링을 위한 함수 생성
col = ['Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [44]:
data_scaled(df, col)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_Yes,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_S,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_Yes,RichPoor_poor,RichPoor_rich,Group_Yes,Gender_male
0,0.715470,-0.334894,-0.279292,-0.304198,-0.272993,-0.266558,1,0,0,1,...,0,0,0,0,1,0,1,0,0,1
1,-0.333218,-0.166089,-0.273660,-0.258609,0.232511,-0.226756,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
2,2.043809,-0.268301,1.958587,-0.304198,5.909999,-0.222233,1,0,0,0,...,0,0,1,0,1,1,0,0,1,1
3,0.295995,-0.334894,0.523616,0.372345,2.792261,-0.091975,1,0,0,0,...,0,0,1,0,1,0,0,0,1,1
4,-0.892518,0.134352,-0.235486,-0.028839,0.247243,-0.264748,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.855295,-0.334894,3.988073,-0.304198,1.239837,-0.199619,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
8689,-0.752693,-0.334894,-0.279292,-0.304198,-0.272993,-0.266558,0,0,1,0,...,1,0,1,1,0,0,1,0,0,0
8690,-0.193393,-0.334894,-0.279292,3.109517,-0.272073,-0.266558,0,0,0,0,...,1,0,1,0,1,0,1,0,0,0
8691,0.226083,-0.334894,0.377178,-0.304198,0.052040,2.659744,1,0,0,0,...,0,0,1,0,0,0,1,0,1,1


In [45]:
data_scaled(dt, col)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_Yes,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_S,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_Yes,RichPoor_poor,RichPoor_rich,Group_Yes,Gender_male
0,-0.115676,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,0,0,1,0,...,1,0,1,0,1,0,1,0,0,0
1,-0.685906,-0.357297,-0.277847,-0.312136,2.287237,-0.246683,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
2,0.169439,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
3,0.668390,-0.357297,4.121036,-0.312136,-0.103990,0.226621,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1
4,-0.614627,-0.340684,-0.283807,0.832025,-0.267810,-0.246683,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.383275,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,0,0,1,0,...,1,0,1,0,1,0,1,0,1,1
4273,0.953505,-0.357297,0.277063,-0.281505,-0.258759,-0.130177,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4274,-0.115676,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,0,1,1,0,...,0,0,0,0,0,0,1,0,0,1
4275,-0.115676,-0.357297,1.490845,-0.312136,-0.267810,0.176459,1,0,0,0,...,0,0,0,0,1,0,1,0,0,1


# 데이터셋 분리

In [46]:
X_train, X_test, y_train, y_test = train_test_split(df, target, shuffle = True, random_state = 109)
X_train, X_val, y_train, y_val = train_test_split(df, target, shuffle = True,\
                                                  test_size=0.15, random_state = 109)

In [47]:
def score_funcs(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차행렬 \n', confusion)
    print('정확도 : {0:.4f}, 정밀도: {1:.4f}, \
         재현율 : {2:.4f}, F1 : {3:.4f}, AUC : {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

# stacking Ensemble
<br>

- 정의 : 다수의 개별 모델이 예측한 데이터 값을 다시 트레이닝 셋으로 사용하여 학습하는 알고리즘이다.
- **이용한 개별 모델**
 > RandomForestClassifier, DecisionTreeClassifier, AdaboostClassifier
  - AdaboostClassifier : AdaBoost는 과적 합되는 경향을 줄이는 데 도움이되고 , 약한 분류기의 실수를 반복적으로 수정, 약한 학습자를 결합하여 정확도를 높이는 알고리즘
  
- **최종 메타 모델**
 > LogisticRegression

## 기본 스태킹
### 개별 머신러닝 모델생성 및 학습

In [48]:
rf_clf = RandomForestClassifier(n_estimators=150, random_state=109)
df_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=150)

# 스태킹 최종 모델
lr_final = LogisticRegression(solver = 'liblinear')

In [49]:
# 개별 학습
rf_clf.fit(X_train, y_train)
df_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=150)

In [50]:
# 개별 모델들의 예측 데이터 생성(train)
Trf_pred = rf_clf.predict(X_train)
Tdt_pred = df_clf.predict(X_train)
Tada_pred = ada_clf.predict(X_train)

# pred_proba
Trf_pred_proba = rf_clf.predict_proba(X_train)[:,1]
Tdt_pred_proba = df_clf.predict_proba(X_train)[:,1]
Tada_pred_proba = ada_clf.predict_proba(X_train)[:,1]

# 모델 별 정확도
print('RF모델 정확도\n', score_funcs(y_train,Trf_pred,Trf_pred_proba))
print('DT모델 정확도\n', score_funcs(y_train,Tdt_pred,Tdt_pred_proba))
print('Ada모델 정확도\n', score_funcs(y_train,Tada_pred,Tada_pred_proba))

오차행렬 
 [[3343  185]
 [  89 3501]]
정확도 : 0.9615, 정밀도: 0.9498,          재현율 : 0.9752, F1 : 0.9623, AUC : 0.9939
RF모델 정확도
 None
오차행렬 
 [[3394  134]
 [ 140 3450]]
정확도 : 0.9615, 정밀도: 0.9626,          재현율 : 0.9610, F1 : 0.9618, AUC : 0.9964
DT모델 정확도
 None
오차행렬 
 [[2678  850]
 [ 577 3013]]
정확도 : 0.7995, 정밀도: 0.7800,          재현율 : 0.8393, F1 : 0.8085, AUC : 0.8905
Ada모델 정확도
 None


In [51]:
# 개별 모델들의 예측 데이터 생성(test)
rf_pred = rf_clf.predict(X_test)
df_pred = df_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)
# pred_proba
rf_pred_proba = rf_clf.predict_proba(X_test)[:,1]
df_pred_proba = df_clf.predict_proba(X_test)[:,1]
ada_pred_proba = ada_clf.predict_proba(X_test)[:,1]

# 모델 별 정확도
print('RF모델 정확도\n', score_funcs(y_test,rf_pred,rf_pred_proba))
print('DT모델 정확도\n', score_funcs(y_test,df_pred,df_pred_proba))
print('Ada모델 정확도\n', score_funcs(y_test,ada_pred,ada_pred_proba))

오차행렬 
 [[901 134]
 [160 899]]
정확도 : 0.8596, 정밀도: 0.8703,          재현율 : 0.8489, F1 : 0.8595, AUC : 0.9267
RF모델 정확도
 None
오차행렬 
 [[841 194]
 [170 889]]
정확도 : 0.8262, 정밀도: 0.8209,          재현율 : 0.8395, F1 : 0.8301, AUC : 0.8477
DT모델 정확도
 None
오차행렬 
 [[797 238]
 [180 879]]
정확도 : 0.8004, 정밀도: 0.7869,          재현율 : 0.8300, F1 : 0.8079, AUC : 0.8848
Ada모델 정확도
 None


In [52]:
# 검증 셋 데이터 용
Tpred = np.array([Trf_pred, Tdt_pred, Tada_pred])
print(Tpred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
Tpred = np.transpose(Tpred)
print(Tpred.shape)

(3, 7118)
(7118, 3)


In [53]:
# 최종 메타 모델 학습 및 예측(검증)
lr_final.fit(Tpred, y_train)
Tfinal = lr_final.predict(Tpred)
Tfinal_proba = lr_final.predict_proba(Tpred)[:,1]
print('최종 메타 모델 정확도 \n',score_funcs(y_train, Tfinal,Tfinal_proba))

오차행렬 
 [[3345  183]
 [  91 3499]]
정확도 : 0.9615, 정밀도: 0.9503,          재현율 : 0.9747, F1 : 0.9623, AUC : 0.9738
최종 메타 모델 정확도 
 None


In [54]:
pred = np.array([rf_pred, df_pred, ada_pred])
print(pred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
pred = np.transpose(pred)
print(pred.shape)

(3, 2094)
(2094, 3)


In [55]:
# 최종 메타 모델 학습 및 예측
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)
final_proba = lr_final.predict_proba(pred)[:,1]
print('최종 메타 모델 정확도 \n',score_funcs(y_test, final, final_proba))

오차행렬 
 [[894 141]
 [154 905]]
정확도 : 0.8591, 정밀도: 0.8652,          재현율 : 0.8546, F1 : 0.8599, AUC : 0.8955
최종 메타 모델 정확도 
 None


- 정확도 자체는 높은 편이지만, 과대적합이 심함

## cv세트 기반 스태킹

In [56]:
# 개별 기반 모델에서 최종 모델이 사용할 학습 검정 데이터 생성 함수
def get_data(model, X_train_n, y_train_n, X_test_n, n_folds):
    # kfold 생성
    kf = KFold(n_splits=n_folds, shuffle=False)
    # 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0],1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작')
    
    for fold_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 기반 모델의 학습 예측 폴드 데이터 셋 추출
        print('\t 폴드세트 :', fold_counter, ' model 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # fitting(in foldsets)
        model.fit(X_tr, y_tr)
        # predicting(in foldsets)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        # predicting((in rawtest_sets))
        test_pred[:,fold_counter] = model.predict(X_test_n)
        
    # make test data using mean of predict data
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1,1)
    
    # 생성된 학습, 테스트 데이터 반환
    return train_fold_pred, test_pred_mean

In [57]:
# 배열로 값 바꾸기
X_train_n = X_train.values
y_train_n = y_train.values
X_test_n = X_test.values

In [58]:
rf_train, rf_test = get_data(rf_clf, X_train_n, y_train_n, X_test_n, 5)
df_train, df_test = get_data(df_clf, X_train_n, y_train_n, X_test_n, 5)
ada_train, ada_test = get_data(ada_clf, X_train_n, y_train_n, X_test_n, 5)

RandomForestClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
DecisionTreeClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
AdaBoostClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작


In [59]:
final_X_train = np.concatenate((rf_train, df_train, ada_train), axis = 1)
final_X_test = np.concatenate(( rf_test, df_test, ada_test), axis =1)
print( '원본 피처 크기(학습, 테스트)', X_train.shape, X_test.shape)
print( '스태킹 피처 크기(학습, 테스트)', final_X_train.shape, final_X_test.shape)

원본 피처 크기(학습, 테스트) (7118, 24) (2094, 24)
스태킹 피처 크기(학습, 테스트) (7118, 3) (2094, 3)


In [60]:
# 최종 모델에 학습
lr_final.fit(final_X_train, y_train)
stack_final = lr_final.predict(final_X_test)
stack_final_proba = lr_final.predict_proba(final_X_test)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_test, stack_final, stack_final_proba))

오차행렬 
 [[846 189]
 [170 889]]
정확도 : 0.8286, 정밀도: 0.8247,          재현율 : 0.8395, F1 : 0.8320, AUC : 0.9021
최종 메타 모델의 예측 정확도 
 None


In [61]:
# train sets
Tstack_final = lr_final.predict(final_X_train)
Tstack_final_proba = lr_final.predict_proba(final_X_train)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_train, Tstack_final, Tstack_final_proba))

오차행렬 
 [[2868  660]
 [ 836 2754]]
정확도 : 0.7898, 정밀도: 0.8067,          재현율 : 0.7671, F1 : 0.7864, AUC : 0.8450
최종 메타 모델의 예측 정확도 
 None


- cv기반으로 진행하니 정확도는 떨어져 보이지만 
- 과대적합이 해소 되고 과소 적합으로 전환

# stacking ensemble & hyperopt

In [62]:
xg_clf = XGBClassifier()
rf_clf = RandomForestClassifier(n_estimators=150, random_state=109)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=150)

# 스태킹 최종 모델
lr_final = LogisticRegression(solver = 'liblinear')

## 개별 모델들 하이퍼 오피티
### GridSearchCV_Knn & Ada

In [63]:
# GridSearchCV, 하이퍼파라미터 조정 값이 작은 모델
ada_grid = {
      'learning_rate' : [0.5,0.65,0.78,0.88,0.95],
    'n_estimators' : [25,50,75,90,100,150]
  }
ada_gridCV = GridSearchCV(ada_clf, param_grid = ada_grid, scoring="accuracy", cv=5, n_jobs=-1, verbose =1)

ada_gridCV.fit(X_train,y_train)
print("최고 평균 정확도 : {}".format(ada_gridCV.best_score_))
print("최고의 파라미터 :", ada_gridCV.best_params_)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
최고 평균 정확도 : 0.7903934755659432
최고의 파라미터 : {'learning_rate': 0.95, 'n_estimators': 90}


In [64]:
# Ada모델링
ada_model = AdaBoostClassifier(n_estimators=90,
                               learning_rate = 0.95).fit(X_train, y_train)
ada_pred = ada_model.predict(X_test)
ada_proba = ada_model.predict_proba(X_test)[:,1]
print('## Ada 정확도 \n', score_funcs(y_test, ada_pred, ada_proba))

오차행렬 
 [[808 227]
 [189 870]]
정확도 : 0.8013, 정밀도: 0.7931,          재현율 : 0.8215, F1 : 0.8071, AUC : 0.8856
## Ada 정확도 
 None


In [65]:
# 학습세트
Tada_pred = ada_model.predict(X_train)
Tada_proba = ada_model.predict_proba(X_train)[:,1]
print('## Ada 정확도(train) \n', score_funcs(y_train, Tada_pred, Tada_proba))

오차행렬 
 [[2711  817]
 [ 619 2971]]
정확도 : 0.7983, 정밀도: 0.7843,          재현율 : 0.8276, F1 : 0.8054, AUC : 0.8876
## Ada 정확도(train) 
 None


In [66]:
# hyperopt 적용을 위한 설정 값 담기
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }
rf_search_space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 12, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50])
                  }
dt_search_space = {'max_depth': hp.quniform('max_depth', 10, 12, 10),
                   'n_estimators': hp.choice('n_estimators', [10,50]),
                    'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1)
                                             }

In [67]:
# 실행을 위한 함수 정의
def xg_objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=50, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

    
def rf_objective_func(rf_search_space):    
    rf_clf = RandomForestClassifier(criterion = rf_search_space['criterion'], 
                                   max_depth = int(rf_search_space['max_depth']),
                                 max_features = rf_search_space['max_features'],
                                 min_samples_leaf = rf_search_space['min_samples_leaf'],
                                 min_samples_split = rf_search_space['min_samples_split'],
                                 n_estimators = rf_search_space['n_estimators'], 
                                 )
    rf_accuracy = cross_val_score(rf_clf, X_train, y_train, cv = 4).mean()
    return { 'loss':-1 *rf_accuracy,'status': STATUS_OK}

def dt_objective_func(dt_search_space):    
    dt_clf = RandomForestClassifier(max_depth = int(dt_search_space['max_depth']),

                                 min_samples_leaf = dt_search_space['min_samples_leaf'],
                                 min_samples_split = dt_search_space['min_samples_split'],
                                 n_estimators = dt_search_space['n_estimators'], 
                                 )
    dt_accuracy = cross_val_score(dt_clf, X_train, y_train, cv = 4).mean()
    return { 'loss':-1 *dt_accuracy,'status': STATUS_OK}


### XGBClassifier의 hyperopt

In [68]:
trial_val = Trials()
xgb_best = fmin(fn=xg_objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', xgb_best)

100%|████████████████████████████████████████████████| 50/50 [00:10<00:00,  4.59trial/s, best loss: -0.806266606688938]
best: {'colsample_bytree': 0.6193829886091825, 'learning_rate': 0.13704847650855984, 'max_depth': 7.0, 'min_child_weight': 1.0}


In [69]:
# 모델링
xg_model = XGBClassifier(n_estimators=200, 
                          learning_rate=round(xgb_best['learning_rate'], 5), 
                            max_depth=int(xgb_best['max_depth']),
                          min_child_weight=int(xgb_best['min_child_weight']),
                            colsample_bytree=round(xgb_best['colsample_bytree'], 5)
                           )
xg_model.fit(X_train, y_train)
xgb_pred= xg_model.predict(X_test)
xgb_pred_proba = xg_model.predict_proba(X_test)[:,1]
print('## XGB, n_estimators=150')
score_funcs(y_test, xgb_pred, xgb_pred_proba)

## XGB, n_estimators=150
오차행렬 
 [[860 175]
 [161 898]]
정확도 : 0.8395, 정밀도: 0.8369,          재현율 : 0.8480, F1 : 0.8424, AUC : 0.9188


In [70]:
# train 세트
Txgb_pred= xg_model.predict(X_train)
Txgb_pred_proba = xg_model.predict_proba(X_train)[:,1]
print('## XGB Train세트, n_estimators=200')
score_funcs(y_train, Txgb_pred, Txgb_pred_proba)

## XGB Train세트, n_estimators=200
오차행렬 
 [[3118  410]
 [ 310 3280]]
정확도 : 0.8988, 정밀도: 0.8889,          재현율 : 0.9136, F1 : 0.9011, AUC : 0.9675


### RFClassifier의 hyper opt

In [71]:
## RandomForest
trials = Trials()
rf_best = fmin(fn= rf_objective_func,
            space= rf_search_space,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)
print('best:', rf_best)

100%|███████████████████████████████████████████████| 50/50 [00:09<00:00,  5.19trial/s, best loss: -0.7655248340501859]
best: {'criterion': 1, 'max_depth': 10.0, 'max_features': 2, 'min_samples_leaf': 0.007076216440627517, 'min_samples_split': 0.05512283752586606, 'n_estimators': 1}


In [72]:
# quniform과 choice의 경우 무작위 형태라 설정값 딕셔너리를 따로 정의
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 75, 3: 100, 4: 125}
# 모델링
rf_model = RandomForestClassifier(criterion = crit[rf_best['criterion']], 
                                      max_depth = int(rf_best['max_depth']), 
                                      max_features = feat[rf_best['max_features']], 
                                      min_samples_leaf = rf_best['min_samples_leaf'], 
                                      min_samples_split = rf_best['min_samples_split'], 
                                      n_estimators = est[rf_best['n_estimators']])
                                    

rf_model.fit(X_train, y_train)
rf_pred= rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:,1]
print('## RFC')
score_funcs(y_test, rf_pred, rf_pred_proba)

## RFC
오차행렬 
 [[865 170]
 [319 740]]
정확도 : 0.7665, 정밀도: 0.8132,          재현율 : 0.6988, F1 : 0.7517, AUC : 0.8632


In [73]:
# train 세트
Trf_pred= rf_model.predict(X_train)
Trf_pred_proba = rf_model.predict_proba(X_train)[:,1]
print('## RFC 학습 세트')
score_funcs(y_train, Trf_pred, Trf_pred_proba)

## RFC 학습 세트
오차행렬 
 [[2911  617]
 [1061 2529]]
정확도 : 0.7643, 정밀도: 0.8039,          재현율 : 0.7045, F1 : 0.7509, AUC : 0.8627


### DTClassifier의 hyperopt

In [74]:
## DecisionTreeClassifier
trials = Trials()
dt_best = fmin(fn= dt_objective_func,
            space= dt_search_space,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)
print('best:', dt_best)

100%|███████████████████████████████████████████████| 50/50 [00:08<00:00,  5.99trial/s, best loss: -0.7441696509211715]
best: {'max_depth': 10.0, 'min_samples_leaf': 0.03456897738636918, 'min_samples_split': 0.004929879505981838, 'n_estimators': 1}


In [75]:
# 모델링
dt_model = RandomForestClassifier(max_depth = int(dt_best['max_depth']), 
                                      min_samples_leaf = dt_best['min_samples_leaf'], 
                                      min_samples_split = dt_best['min_samples_split'], 
                                      n_estimators = est[dt_best['n_estimators']])
                                    

dt_model.fit(X_train, y_train)
dt_pred= dt_model.predict(X_test)
dt_pred_proba = dt_model.predict_proba(X_test)[:,1]
print('## DTC')
score_funcs(y_test, dt_pred, dt_pred_proba)

## DTC
오차행렬 
 [[853 182]
 [342 717]]
정확도 : 0.7498, 정밀도: 0.7976,          재현율 : 0.6771, F1 : 0.7324, AUC : 0.8553


In [76]:
# train 세트
Tdt_pred= dt_model.predict(X_train)
Tdt_pred_proba = dt_model.predict_proba(X_train)[:,1]
print('## DTC train sets')
score_funcs(y_train, Tdt_pred, Tdt_pred_proba)

## DTC train sets
오차행렬 
 [[2867  661]
 [1158 2432]]
정확도 : 0.7445, 정밀도: 0.7863,          재현율 : 0.6774, F1 : 0.7278, AUC : 0.8472


## 최종 모델 용

In [77]:
lr_pred = np.array([xgb_pred, rf_pred, dt_pred, ada_pred])
print(lr_pred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
lr_pred = np.transpose(lr_pred)
print(lr_pred.shape)

(4, 2094)
(2094, 4)


In [78]:
# 최종 메타 모델 학습 및 예측
lr_final.fit(lr_pred, y_test)
final = lr_final.predict(lr_pred)
final_proba = lr_final.predict_proba(lr_pred)[:,1]
print('최종 메타 모델 정확도 \n',score_funcs(y_test, final, final_proba))

오차행렬 
 [[860 175]
 [161 898]]
정확도 : 0.8395, 정밀도: 0.8369,          재현율 : 0.8480, F1 : 0.8424, AUC : 0.8700
최종 메타 모델 정확도 
 None


In [79]:
# 검증 셋 데이터 용
Tpred = np.array([Txgb_pred, Trf_pred, Tdt_pred, Tada_pred])
print(Tpred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
Tpred = np.transpose(Tpred)
print(Tpred.shape)

(4, 7118)
(7118, 4)


In [80]:
# 최종 메타 모델 학습 및 예측(검증)
lr_final.fit(Tpred, y_train)
Tfinal = lr_final.predict(Tpred)
Tfinal_proba = lr_final.predict_proba(Tpred)[:,1]
print('최종 메타 모델 정확도 \n',score_funcs(y_train, Tfinal,Tfinal_proba))

오차행렬 
 [[3118  410]
 [ 310 3280]]
정확도 : 0.8988, 정밀도: 0.8889,          재현율 : 0.9136, F1 : 0.9011, AUC : 0.9060
최종 메타 모델 정확도 
 None


- 하이퍼오피티를 실시하여 최적화를 구했더니 정확도도 올라가고 기존스태킹 앙상블보다는 과대적합이 적은편

## 최종모델 cv기반 스태킹

In [81]:
# 배열로 값 바꾸기
X_train_n = X_train.values
y_train_n = y_train.values
X_test_n = X_test.values

In [82]:
rf_train, rf_test = get_data(rf_clf, X_train_n, y_train_n, X_test_n, 5)
df_train, df_test = get_data(df_clf, X_train_n, y_train_n, X_test_n, 5)
ada_train, ada_test = get_data(ada_clf, X_train_n, y_train_n, X_test_n, 5)

RandomForestClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
DecisionTreeClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
AdaBoostClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작


In [83]:
final_X_train = np.concatenate((rf_train, df_train, ada_train), axis = 1)
final_X_test = np.concatenate(( rf_test, df_test, ada_test), axis =1)
print( '원본 피처 크기(학습, 테스트)', X_train.shape, X_test.shape)
print( '스태킹 피처 크기(학습, 테스트)', final_X_train.shape, final_X_test.shape)

원본 피처 크기(학습, 테스트) (7118, 24) (2094, 24)
스태킹 피처 크기(학습, 테스트) (7118, 3) (2094, 3)


In [84]:
# 최종 모델에 학습
lr_final.fit(final_X_train, y_train)
stack_final = lr_final.predict(final_X_test)
stack_final_proba = lr_final.predict_proba(final_X_test)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_test, stack_final, stack_final_proba))

오차행렬 
 [[846 189]
 [171 888]]
정확도 : 0.8281, 정밀도: 0.8245,          재현율 : 0.8385, F1 : 0.8315, AUC : 0.9025
최종 메타 모델의 예측 정확도 
 None


In [85]:
# train sets
Tstack_final = lr_final.predict(final_X_train)
Tstack_final_proba = lr_final.predict_proba(final_X_train)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_train, Tstack_final, Tstack_final_proba))

오차행렬 
 [[2867  661]
 [ 831 2759]]
정확도 : 0.7904, 정밀도: 0.8067,          재현율 : 0.7685, F1 : 0.7872, AUC : 0.8451
최종 메타 모델의 예측 정확도 
 None


# 개별 모델 대체한 스태킹 앙상블
<br>

- 정확도를 조금 더 올리기 위해 AdaboostClassifier모델을 제거하고, LGBMClassifier 모델을 사용함
- LGBMClassifier
 >  속도가 빠르고 데이터 수가 많을 때 편리한 알고리즘이다. <br>
 다른 트리기반 알고리즘과 달리 트리가 수평적으로 확장한다는 차이점이 있다.


In [90]:
xgb_clf = xgb.XGBClassifier()
lgbm_clf = LGBMClassifier(n_estimators=150, random_state=109)
rf_clf = RandomForestClassifier(n_estimators=150, random_state=109)
df_clf = DecisionTreeClassifier()

# 스태킹 최종 모델
lr_final = LogisticRegression(solver = 'liblinear')

In [95]:
# 개별 학습
xgb_clf.fit(X_train, y_train)
lgbm_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
df_clf.fit(X_train, y_train)

# 개별 모델들의 예측 데이터 생성(test)
xgb_pred = xgb_clf.predict(X_test)
lgbm_pred = lgbm_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
df_pred = df_clf.predict(X_test)
# pred_proba
xgb_pred_proba = xgb_clf.predict_proba(X_test)[:,1]
lgbm_pred_proba = lgbm_clf.predict_proba(X_test)[:,1]
rf_pred_proba = rf_clf.predict_proba(X_test)[:,1]
df_pred_proba = df_clf.predict_proba(X_test)[:,1]

# 모델 별 정확도
print('XG모델 정확도\n', score_funcs(y_test,xgb_pred,xgb_pred_proba))
print('LG모델 정확도\n', score_funcs(y_test,lgbm_pred,lgbm_pred_proba))
print('RF모델 정확도\n', score_funcs(y_test,rf_pred,rf_pred_proba))
print('DT모델 정확도\n', score_funcs(y_test,df_pred,df_pred_proba))

오차행렬 
 [[871 164]
 [161 898]]
정확도 : 0.8448, 정밀도: 0.8456,          재현율 : 0.8480, F1 : 0.8468, AUC : 0.9213
XG모델 정확도
 None
오차행렬 
 [[854 181]
 [166 893]]
정확도 : 0.8343, 정밀도: 0.8315,          재현율 : 0.8432, F1 : 0.8373, AUC : 0.9194
LG모델 정확도
 None
오차행렬 
 [[901 134]
 [160 899]]
정확도 : 0.8596, 정밀도: 0.8703,          재현율 : 0.8489, F1 : 0.8595, AUC : 0.9267
RF모델 정확도
 None
오차행렬 
 [[847 188]
 [175 884]]
정확도 : 0.8266, 정밀도: 0.8246,          재현율 : 0.8347, F1 : 0.8297, AUC : 0.8482
DT모델 정확도
 None


In [97]:
pred = np.array([xgb_pred, lgbm_pred, rf_pred, df_pred])
print(pred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
pred = np.transpose(pred)
print(pred.shape)

(4, 2094)
(2094, 4)


In [98]:
# 최종 메타 모델 학습 및 예측
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)
final_proba = lr_final.predict_proba(pred)[:,1]
print('최종 메타 모델 정확도 \n',score_funcs(y_test, final, final_proba))

오차행렬 
 [[900 135]
 [156 903]]
정확도 : 0.8610, 정밀도: 0.8699,          재현율 : 0.8527, F1 : 0.8612, AUC : 0.8973
최종 메타 모델 정확도 
 None


In [99]:
# train sets
Tpred = np.array([Txgb_pred, Tlgbm_pred, Trf_pred, Tdf_pred])
print(Tpred.shape)
Tpred = np.transpose(Tpred)
print(Tpred.shape)

# fitting & predict
lr_final.fit(Tpred, y_train)
Tfinal = lr_final.predict(Tpred)
Tfinal_proba = lr_final.predict_proba(Tpred)[:,1]
print('최종 메타 모델 정확도(train) \n',score_funcs(y_train, Tfinal, Tfinal_proba))

(4, 7118)
(7118, 4)
오차행렬 
 [[3348  180]
 [  94 3496]]
정확도 : 0.9615, 정밀도: 0.9510,          재현율 : 0.9738, F1 : 0.9623, AUC : 0.9749
최종 메타 모델 정확도(train) 
 None


## cv 기반

In [100]:
# 배열로 값 바꾸기
X_train_n = X_train.values
y_train_n = y_train.values
X_test_n = X_test.values

In [101]:
# 함수는 6.2에 정의

xgb_train, xgb_test = get_data(xgb_clf, X_train_n, y_train_n, X_test_n, 5)
lgbm_train,lgbm_test = get_data(lgbm_clf, X_train_n, y_train_n, X_test_n, 5)
rf_train, rf_test = get_data(rf_clf, X_train_n, y_train_n, X_test_n, 5)
df_train, df_test = get_data(df_clf, X_train_n, y_train_n, X_test_n, 5)

XGBClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
LGBMClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
RandomForestClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
DecisionTreeClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작


In [102]:
model_X_train = np.concatenate((xgb_train, lgbm_train, rf_train, df_train), axis = 1)
model_X_test = np.concatenate((xgb_test, lgbm_test, rf_test, df_test), axis =1)
print( '원본 피처 크기(학습, 테스트)', X_train.shape, X_test.shape)
print( '스태킹 피처 크기(학습, 테스트)', model_X_train.shape, model_X_test.shape)

원본 피처 크기(학습, 테스트) (7118, 24) (2094, 24)
스태킹 피처 크기(학습, 테스트) (7118, 4) (2094, 4)


In [103]:
# 최종 모델에 학습
lr_final.fit(model_X_train, y_train)
stack_final = lr_final.predict(model_X_test)
stack_final_proba = lr_final.predict_proba(model_X_test)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_test, stack_final, stack_final_proba))

오차행렬 
 [[866 169]
 [150 909]]
정확도 : 0.8477, 정밀도: 0.8432,          재현율 : 0.8584, F1 : 0.8507, AUC : 0.9076
최종 메타 모델의 예측 정확도 
 None


In [104]:
# train sets
Tstack_final = lr_final.predict(model_X_train)
Tstack_final_proba = lr_final.predict_proba(model_X_train)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_train, Tstack_final, Tstack_final_proba))

오차행렬 
 [[2854  674]
 [ 723 2867]]
정확도 : 0.8037, 정밀도: 0.8097,          재현율 : 0.7986, F1 : 0.8041, AUC : 0.8445
최종 메타 모델의 예측 정확도 
 None


- 과대적합은 해소되었으나 과소적합을 의심할 만함

## hyperopt 적용

In [105]:
## 개별 모델 정의
xgb_clf = XGBClassifier()
lgbm_clf = LGBMClassifier(n_estimators=150, random_state=109)
rf_clf = RandomForestClassifier(n_estimators=150, random_state=109)
df_clf = DecisionTreeClassifier()

# 스태킹 최종 모델
lr_final = LogisticRegression(solver = 'liblinear')

### hyperopt 스페이스 조정

In [106]:
# hyperopt 적용을 위한 설정 값 담기
xgb_search_space = {'n_estimators': hp.choice('n_estimators', [10,50]),
                    'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }

lgbm_search_space = {'num_leaves':hp.quniform('num_leaves', 32, 64, 1),
    'max_depth':hp.quniform('max_depth', 100, 160, 1),
    'min_child_samples':hp.quniform('min_child_samples', 60,100, 1),
    'subsample':hp.uniform('subsample', 0.7, 1),
    'learning_rate':hp.uniform('learning_rate', 0.01, 0.2)
}

rf_search_space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 12, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50])
                  }
dt_search_space = {'max_depth': hp.quniform('max_depth', 10, 12, 10),
                   'n_estimators': hp.choice('n_estimators', [10,50]),
                    'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1)
                                             }

In [107]:
# 목적함수 정의
def xgb_objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators = search_space['n_estimators'],
                            max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    xgb_accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(xgb_accuracy), 'status': STATUS_OK}

def lgbm_objective_funcs(search_space):
    lgbm_clf = LGBMClassifier(n_estimators=100, num_leaves = int(search_space['num_leaves']),
                              max_depth = int(search_space['max_depth']),
                              min_child_samples = int(search_space['min_child_samples']),
                              subsample = search_space['subsample'],
                              learning_rate = search_space['learning_rate']
                             )
    lgbm_accuracy = cross_val_score(lgbm_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(lgbm_accuracy), 'status': STATUS_OK}
    
def rf_objective_func(rf_search_space):    
    rf_clf = RandomForestClassifier(criterion = rf_search_space['criterion'], 
                                   max_depth = int(rf_search_space['max_depth']),
                                 max_features = rf_search_space['max_features'],
                                 min_samples_leaf = rf_search_space['min_samples_leaf'],
                                 min_samples_split = rf_search_space['min_samples_split'],
                                 n_estimators = rf_search_space['n_estimators'], 
                                 )
    rf_accuracy = cross_val_score(rf_clf, X_train, y_train, cv = 4).mean()
    return { 'loss':-1 *rf_accuracy,'status': STATUS_OK}

def dt_objective_func(dt_search_space):    
    dt_clf = RandomForestClassifier(max_depth = int(dt_search_space['max_depth']),
                                 min_samples_leaf = dt_search_space['min_samples_leaf'],
                                 min_samples_split = dt_search_space['min_samples_split'],
                                 n_estimators = dt_search_space['n_estimators'], 
                                 )
    dt_accuracy = cross_val_score(dt_clf, X_train, y_train, cv = 4).mean()
    return { 'loss':-1 *dt_accuracy,'status': STATUS_OK}

### XGBClassifier

In [108]:
trial_val = Trials()
xgb_best = fmin(fn=xgb_objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', xgb_best)

100%|███████████████████████████████████████████████| 50/50 [00:10<00:00,  4.74trial/s, best loss: -0.8026138161493114]
best: {'colsample_bytree': 0.8418286288021363, 'learning_rate': 0.04801060921231198, 'max_depth': 14.0, 'min_child_weight': 2.0, 'n_estimators': 1}


In [109]:
est = {0: 10, 1: 50, 2: 75, 3: 100, 4: 125}
# 모델링
xgb_model = XGBClassifier(n_estimators = est[xgb_best['n_estimators']],
                          learning_rate=round(xgb_best['learning_rate'], 5), 
                            max_depth=int(xgb_best['max_depth']),
                          min_child_weight=int(xgb_best['min_child_weight']),
                            colsample_bytree=round(xgb_best['colsample_bytree'], 5)
                           )
xgb_model.fit(X_train, y_train)
xg_pred= xgb_model.predict(X_test)
xg_pred_proba = xgb_model.predict_proba(X_test)[:,1]
print('## XGB')
score_funcs(y_test, xg_pred, xg_pred_proba)

## XGB
오차행렬 
 [[861 174]
 [161 898]]
정확도 : 0.8400, 정밀도: 0.8377,          재현율 : 0.8480, F1 : 0.8428, AUC : 0.9171


In [110]:
# train sets
Txg_pred= xgb_model.predict(X_train)
Txg_pred_proba = xgb_model.predict_proba(X_train)[:,1]
print('## XGB(train)')
score_funcs(y_train, Txg_pred, Txg_pred_proba)

## XGB(train)
오차행렬 
 [[3131  397]
 [ 338 3252]]
정확도 : 0.8967, 정밀도: 0.8912,          재현율 : 0.9058, F1 : 0.8985, AUC : 0.9651


### LGBMClassifier

In [111]:
trials =Trials()

lgbm_best = fmin(fn = lgbm_objective_funcs, 
            space = lgbm_search_space, 
            algo = tpe.suggest, 
           max_evals = 50, trials = trials, 
            rstate = np.random.default_rng(seed=9))

print('best: ', lgbm_best)

100%|███████████████████████████████████████████████| 50/50 [00:13<00:00,  3.83trial/s, best loss: -0.8031756928173829]
best:  {'learning_rate': 0.04629077656287839, 'max_depth': 119.0, 'min_child_samples': 69.0, 'num_leaves': 32.0, 'subsample': 0.9835048715830431}


In [112]:
lgbm_model = LGBMClassifier(n_estimators = 100, 
                            num_leaves = int(lgbm_best['num_leaves']),
                              max_depth = int(lgbm_best['max_depth']),
                              min_child_samples = int(lgbm_best['min_child_samples']),
                              subsample = round(lgbm_best['subsample'], 5),
                              learning_rate = round(lgbm_best['learning_rate'], 5))

lgbm_model.fit(X_train, y_train)
lgbm_pred = lgbm_model.predict(X_test)
lgbm_pred_proba = lgbm_model.predict_proba(X_test)[:,1]
print('## LGBM')
score_funcs(y_test, lgbm_pred, lgbm_pred_proba)

## LGBM
오차행렬 
 [[826 209]
 [174 885]]
정확도 : 0.8171, 정밀도: 0.8090,          재현율 : 0.8357, F1 : 0.8221, AUC : 0.9024


In [113]:
# train sets
Tlgbm_pred = lgbm_model.predict(X_train)
Tlgbm_pred_proba = lgbm_model.predict_proba(X_train)[:,1]
print('## LGBM(train)')
score_funcs(y_train, Tlgbm_pred, Tlgbm_pred_proba)

## LGBM(train)
오차행렬 
 [[2923  605]
 [ 566 3024]]
정확도 : 0.8355, 정밀도: 0.8333,          재현율 : 0.8423, F1 : 0.8378, AUC : 0.9236


### RandomForestClassifier

In [114]:
## RandomForest
trials = Trials()
rf_best = fmin(fn= rf_objective_func,
            space= rf_search_space,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)
print('best:', rf_best)

100%|███████████████████████████████████████████████| 50/50 [00:11<00:00,  4.28trial/s, best loss: -0.7784494982031314]
best: {'criterion': 1, 'max_depth': 10.0, 'max_features': 3, 'min_samples_leaf': 0.00304706348800704, 'min_samples_split': 0.09645314074879459, 'n_estimators': 1}


In [115]:
# quniform과 choice의 경우 무작위 형태라 설정값 딕셔너리를 따로 정의
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 75, 3: 100, 4: 125}
# 모델링
rf_model = RandomForestClassifier(criterion = crit[rf_best['criterion']], 
                                      max_depth = int(rf_best['max_depth']), 
                                      max_features = feat[rf_best['max_features']], 
                                      min_samples_leaf = rf_best['min_samples_leaf'], 
                                      min_samples_split = rf_best['min_samples_split'], 
                                      n_estimators = est[rf_best['n_estimators']])
                                    

rf_model.fit(X_train, y_train)
rf_pred= rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:,1]
print('## RFC')
score_funcs(y_test, rf_pred, rf_pred_proba)

## RFC
오차행렬 
 [[779 256]
 [192 867]]
정확도 : 0.7861, 정밀도: 0.7720,          재현율 : 0.8187, F1 : 0.7947, AUC : 0.8679


In [116]:
Trf_pred= rf_model.predict(X_train)
Trf_pred_proba = rf_model.predict_proba(X_train)[:,1]
print('## RFC(train)')
score_funcs(y_train, Trf_pred, Trf_pred_proba)

## RFC(train)
오차행렬 
 [[2635  893]
 [ 645 2945]]
정확도 : 0.7839, 정밀도: 0.7673,          재현율 : 0.8203, F1 : 0.7929, AUC : 0.8713


### DecisionTreeClassifier

In [117]:
## DecisionTreeClassifier
trials = Trials()
dt_best = fmin(fn= dt_objective_func,
            space= dt_search_space,
            algo= tpe.suggest,
            max_evals = 50,
            trials= trials)
print('best:', dt_best)

100%|███████████████████████████████████████████████| 50/50 [00:09<00:00,  5.54trial/s, best loss: -0.7967118725960172]
best: {'max_depth': 10.0, 'min_samples_leaf': 0.00013961619288110037, 'min_samples_split': 0.006223963619069704, 'n_estimators': 1}


In [118]:
# 모델링
dt_model = RandomForestClassifier(max_depth = int(dt_best['max_depth']), 
                                      min_samples_leaf = dt_best['min_samples_leaf'], 
                                      min_samples_split = dt_best['min_samples_split'], 
                                      n_estimators = est[dt_best['n_estimators']])
                                    

dt_model.fit(X_train, y_train)
dt_pred= dt_model.predict(X_test)
dt_pred_proba = dt_model.predict_proba(X_test)[:,1]
print('## DTC')
score_funcs(y_test, dt_pred, dt_pred_proba)

## DTC
오차행렬 
 [[826 209]
 [185 874]]
정확도 : 0.8118, 정밀도: 0.8070,          재현율 : 0.8253, F1 : 0.8161, AUC : 0.8945


In [119]:
# train 세트
Tdt_pred= dt_model.predict(X_train)
Tdt_pred_proba = dt_model.predict_proba(X_train)[:,1]
print('## DTC train sets')
score_funcs(y_train, Tdt_pred, Tdt_pred_proba)

## DTC train sets
오차행렬 
 [[2868  660]
 [ 595 2995]]
정확도 : 0.8237, 정밀도: 0.8194,          재현율 : 0.8343, F1 : 0.8268, AUC : 0.9111


### 최종 모델
- DT, RF는 하이퍼를 돌릴수록 정확도가 더 떨어저서 기본모델로 가려 했으나
- 정확도는 떨어지지만 과적합이 많이 감소되어서 그냥 하이퍼적용한 값으로 고

In [120]:
lr_pred = np.array([xgb_pred, lgbm_pred, rf_pred, dt_pred])
print(lr_pred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
lr_pred = np.transpose(lr_pred)
print(lr_pred.shape)

(4, 2094)
(2094, 4)


In [121]:
# 최종 메타 모델 학습 및 예측
lr_final.fit(lr_pred, y_test)
final = lr_final.predict(lr_pred)
final_proba = lr_final.predict_proba(lr_pred)[:,1]
print('최종 메타 모델 정확도 \n',score_funcs(y_test, final, final_proba))

오차행렬 
 [[871 164]
 [161 898]]
정확도 : 0.8448, 정밀도: 0.8456,          재현율 : 0.8480, F1 : 0.8468, AUC : 0.8686
최종 메타 모델 정확도 
 None


In [122]:
# 훈련 세트
Tlr_pred = np.array([Txg_pred, Tlgbm_pred, Trf_pred, Tdt_pred])
print(Tlr_pred.shape)
Tlr_pred = np.transpose(Tlr_pred)
print(Tlr_pred.shape)

# 최종 메타 모델 학습 및 예측(train)
lr_final.fit(Tlr_pred, y_train)
Tfinal = lr_final.predict(Tlr_pred)
Tfinal_proba = lr_final.predict_proba(Tlr_pred)[:,1]
print('최종 메타 모델 정확도(train) \n',score_funcs(y_train, Tfinal, Tfinal_proba))

(4, 7118)
(7118, 4)
오차행렬 
 [[3131  397]
 [ 338 3252]]
정확도 : 0.8967, 정밀도: 0.8912,          재현율 : 0.9058, F1 : 0.8985, AUC : 0.8983
최종 메타 모델 정확도(train) 
 None


- 이전 스태킹 앙상블 모델의 정확도가 시험셋(86%), 학습셋(96%)인 데 반해
- 과대적합이 많이 감소된 것을 확인함.

#### transported예측하기

In [ ]:
# 저장파일 불러오기
#last = pd.read_csv('C:/Users/admin/Downloads/spaceship-titanic/sample_submission.csv')
#last.info()

In [ ]:
# 미리 Transported 컬럼 삭제
#last = last.drop('Transported', axis=1)

In [123]:
# 개별 모델로 타깃값 예측
XGB_transported = xgb_model.predict(dt)
lgbm_transported = lgbm_model.predict(dt)
rf_transported = rf_model.predict(dt)
dt_transported = dt_model.predict(dt)

In [124]:
pred_transported = np.array([XGB_transported, lgbm_transported, rf_transported, dt_transported])
print(pred_transported.shape)

pred_transported = np.transpose(pred_transported)
print(pred_transported.shape)

(4, 4277)
(4277, 4)


In [ ]:
# 완성된 모델로 값 예측 하여 컬럼에 추가
#last['Transported'] = lr_final.predict(pred_transported)

In [ ]:
# 확인
#last.head()

In [ ]:
# csv파일로 내보내기
#last.to_csv('C:/Users/admin/Desktop/sparta/submit5.csv', index=False)

### 최종모델을 이용한 cv기반 스태킹앙상블

In [125]:
# 배열로 값 바꾸기
X_train_n = X_train.values
y_train_n = y_train.values
X_test_n = X_test.values

In [126]:
# 함수는 6.2에 정의

xgb_train, xgb_test = get_data(xgb_clf, X_train_n, y_train_n, X_test_n, 5)
lgbm_train,lgbm_test = get_data(lgbm_clf, X_train_n, y_train_n, X_test_n, 5)
rf_train, rf_test = get_data(rf_clf, X_train_n, y_train_n, X_test_n, 5)
dt_train, dt_test = get_data(dt_clf, X_train_n, y_train_n, X_test_n, 5)

XGBClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
LGBMClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
RandomForestClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작
DecisionTreeClassifier model 시작
	 폴드세트 : 0  model 시작
	 폴드세트 : 1  model 시작
	 폴드세트 : 2  model 시작
	 폴드세트 : 3  model 시작
	 폴드세트 : 4  model 시작


In [127]:
Train = np.concatenate((xgb_train, lgbm_train, rf_train, dt_train), axis = 1)
Test = np.concatenate((xgb_test, lgbm_test, rf_test, df_test), axis =1)
print( '원본 피처 크기(학습, 테스트)', X_train.shape, X_test.shape)
print( '스태킹 피처 크기(학습, 테스트)', Train.shape, Test.shape)

원본 피처 크기(학습, 테스트) (7118, 24) (2094, 24)
스태킹 피처 크기(학습, 테스트) (7118, 4) (2094, 4)


In [128]:
# 최종 모델에 학습
lr_final.fit(Train, y_train)
model_pred = lr_final.predict(Test)
model_pred_proba = lr_final.predict_proba(Test)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_test, model_pred, model_pred_proba))

오차행렬 
 [[866 169]
 [149 910]]
정확도 : 0.8481, 정밀도: 0.8434,          재현율 : 0.8593, F1 : 0.8513, AUC : 0.9076
최종 메타 모델의 예측 정확도 
 None


In [129]:
# train sets
Tmodel_pred = lr_final.predict(Train)
Tmodel_pred_proba = lr_final.predict_proba(Train)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_train, Tmodel_pred, Tmodel_pred_proba))

오차행렬 
 [[2854  674]
 [ 723 2867]]
정확도 : 0.8037, 정밀도: 0.8097,          재현율 : 0.7986, F1 : 0.8041, AUC : 0.8443
최종 메타 모델의 예측 정확도 
 None
